<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# -*- coding: utf-8 -*-
"""
================================================================================
NOTEBOOK 3 - FASE 1: SISTEMA DE ÍNDICE INTELIGENTE
================================================================================
Trabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación
en Fotografía de Retrato

Autor: Jesús L.
Universidad: Universidad Oberta de Cataluña (UOC)
Fecha: Noviembre 2025

CONTEXTO:
Sistema de índice centralizado que NO duplica datos. Mantiene máscaras y
características en sus ubicaciones originales, creando un índice maestro
con rutas y resúmenes ligeros para acceso eficiente en fases posteriores.

- Búsqueda recursiva COMPLETA sin límites de nivel
- Guardado INCREMENTAL después de cada fotografía
- Sistema de checkpoint para recuperación

Referencias:
- Gonzalez, R.C. & Woods, R.E. (2018). Digital Image Processing (4th ed.)
- Lin, T.Y., et al. (2014). Microsoft COCO: Common Objects in Context
- Kirillov, A., et al. (2023). Segment Anything
================================================================================
"""

'\n================================================================================\nNOTEBOOK 3 - FASE 1: SISTEMA DE ÍNDICE INTELIGENTE\n================================================================================\nTrabajo Fin de Máster: Evaluación Comparativa de Técnicas de Segmentación\nen Fotografía de Retrato\n\nAutor: Jesús L.\nUniversidad: Universidad Oberta de Cataluña (UOC)\nFecha: Noviembre 2025\n\nCONTEXTO:\nSistema de índice centralizado que NO duplica datos. Mantiene máscaras y\ncaracterísticas en sus ubicaciones originales, creando un índice maestro\ncon rutas y resúmenes ligeros para acceso eficiente en fases posteriores.\n\n- Búsqueda recursiva COMPLETA sin límites de nivel\n- Guardado INCREMENTAL después de cada fotografía\n- Sistema de checkpoint para recuperación\n\nReferencias:\n- Gonzalez, R.C. & Woods, R.E. (2018). Digital Image Processing (4th ed.)\n- Lin, T.Y., et al. (2014). Microsoft COCO: Common Objects in Context\n- Kirillov, A., et al. (2023). Segment An

In [21]:
import os
import gc
import json
import logging
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field, asdict

import numpy as np
from PIL import Image

In [22]:
# =============================================================================
# SETUP GOOGLE COLAB
# =============================================================================

try:
    import google.colab
    IN_COLAB = True
    print("\nEntorno: Google Colab detectado")
except ImportError:
    IN_COLAB = False
    print("\nEntorno: Ejecución local (no Colab)")

if IN_COLAB:
    print("\nMontando Google Drive...")
    print("-"*80)

    from google.colab import drive

    try:
        drive.mount('/content/drive', force_remount=False)
        print("Google Drive montado correctamente en: /content/drive")

        ruta_tfm_check = Path("/content/drive/MyDrive/TFM")
        if ruta_tfm_check.exists():
            print(f"Directorio TFM encontrado: {ruta_tfm_check}")
        else:
            print(f"\nADVERTENCIA: No se encontró directorio TFM en {ruta_tfm_check}")
            print("Verifica que la estructura del proyecto esté correctamente ubicada")

    except Exception as e:
        print(f"ERROR montando Google Drive: {e}")
        print("Verifica permisos y conexión a Drive")
        raise
else:
    print("\nEjecución local - saltando montaje de Drive")
    print("-"*80)

print("\n")


Entorno: Google Colab detectado

Montando Google Drive...
--------------------------------------------------------------------------------
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado correctamente en: /content/drive
Directorio TFM encontrado: /content/drive/MyDrive/TFM




In [23]:
# =============================================================================
# CONFIGURACIÓN DE LOGGING
# =============================================================================

def configurar_logging() -> logging.Logger:
    """Configura sistema de logging para la Fase 1."""
    logger = logging.getLogger('Fase1_IndiceInteligente')

    if logger.hasHandlers():
        logger.handlers.clear()

    logger.setLevel(logging.INFO)

    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    formatter = logging.Formatter(
        '[%(asctime)s] %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_handler.setFormatter(formatter)

    logger.addHandler(console_handler)
    logger.propagate = False

    return logger


logger = configurar_logging()

In [24]:
# =============================================================================
# CONFIGURACIÓN DE RUTAS
# =============================================================================

@dataclass
class ConfiguracionRutas:
    """Configuración centralizada de rutas del proyecto."""
    ruta_base: Path
    ruta_imagenes: Path
    ruta_caracteristicas: Path
    ruta_gt: Path
    ruta_modelos: Path
    ruta_analisis: Path

    @classmethod
    def descubrir_desde_base(cls, ruta_base: str) -> 'ConfiguracionRutas':
        """Descubre automáticamente la estructura de directorios en TFM."""
        base = Path(ruta_base)

        if not base.exists():
            raise FileNotFoundError(f"Directorio base no existe: {base}")

        logger.info(f"Escaneando estructura en: {base}")

        def encontrar_dir(patron: str, critico: bool = True) -> Optional[Path]:
            directorios = list(base.glob(patron))
            if len(directorios) == 0:
                if critico:
                    raise FileNotFoundError(f"No se encontró directorio crítico: {patron} en {base}")
                return None
            return directorios[0]

        ruta_imagenes = encontrar_dir("0_Imagenes*", critico=True)
        ruta_caracteristicas = encontrar_dir("1_Caracteristicas*", critico=True)

        ruta_cvat = encontrar_dir("0_Imagenes_CVAT*", critico=False)
        if ruta_cvat:
            ruta_gt = ruta_cvat / "ground_truth_masks"
            if not ruta_gt.exists():
                posibles_gt = list(ruta_cvat.glob("*mask*"))
                if posibles_gt:
                    ruta_gt = posibles_gt[0]
                else:
                    raise FileNotFoundError(f"No se encontró subdirectorio ground_truth_masks en {ruta_cvat}")
        else:
            raise FileNotFoundError("No se encontró directorio de ground truth CVAT")

        ruta_modelos = encontrar_dir("2_Modelos*", critico=True)
        ruta_analisis = encontrar_dir("3_Analisis*", critico=False)

        if ruta_analisis is None:
            ruta_analisis = base / "3_Analisis"
            ruta_analisis.mkdir(exist_ok=True, parents=True)
            logger.info(f"  Creado directorio de análisis: {ruta_analisis}")

        return cls(
            ruta_base=base,
            ruta_imagenes=ruta_imagenes,
            ruta_caracteristicas=ruta_caracteristicas,
            ruta_gt=ruta_gt,
            ruta_modelos=ruta_modelos,
            ruta_analisis=ruta_analisis
        )

    def validar(self) -> Tuple[bool, List[str]]:
        """Valida que todas las rutas críticas existan."""
        errores = []

        rutas_criticas = {
            'Imágenes': self.ruta_imagenes,
            'Características': self.ruta_caracteristicas,
            'Ground Truth': self.ruta_gt,
            'Modelos': self.ruta_modelos,
            'Análisis': self.ruta_analisis
        }

        for nombre, ruta in rutas_criticas.items():
            if not ruta.exists():
                errores.append(f"{nombre}: {ruta} no existe")

        return (len(errores) == 0, errores)

    def mostrar_estructura(self):
        """Muestra la estructura descubierta."""
        logger.info("\nEstructura del proyecto:")
        logger.info(f"  Base:             {self.ruta_base}")
        logger.info(f"  Imágenes:         {self.ruta_imagenes.name}")
        logger.info(f"  Características:  {self.ruta_caracteristicas.name}")
        logger.info(f"  Ground Truth:     {self.ruta_gt.parent.name}/{self.ruta_gt.name}")
        logger.info(f"  Modelos:          {self.ruta_modelos.name}")
        logger.info(f"  Análisis:         {self.ruta_analisis.name}")

In [25]:
# =============================================================================
# ESTRUCTURA DE DATOS
# =============================================================================

@dataclass
class IndiceFotografia:
    """Índice de una fotografía con rutas y resúmenes ligeros."""
    foto_id: str
    rutas: Dict[str, str]
    resumen_caracteristicas: Dict[str, Any]
    modelos_disponibles: Dict[str, Dict[str, Any]]
    ground_truth_info: Dict[str, Any]
    timestamp_indexacion: str

    def a_dict(self) -> Dict[str, Any]:
        """Convierte a diccionario serializable."""
        return asdict(self)

In [27]:
# =============================================================================
# EXTRACTOR DE RESÚMENES DE CARACTERÍSTICAS
# =============================================================================

class ExtractorResumenCaracteristicas:
    """Extrae resúmenes ligeros de características fotográficas."""

    @staticmethod
    def extraer_resumen(caracteristicas_completas: Dict[str, Any]) -> Dict[str, Any]:
        """Extrae campos clave de características para el resumen."""
        exif = caracteristicas_completas.get('metadatos_exif', {})
        calidad = caracteristicas_completas.get('calidad', {})
        color = caracteristicas_completas.get('estadisticas_color', {})
        saliencia = caracteristicas_completas.get('saliencia_visual', {})
        metadatos = caracteristicas_completas.get('metadatos_archivo', {})

        resumen = {
            'exif': {
                'iso': exif.get('iso'),
                'apertura_fnumber': exif.get('apertura_fnumber'),
                'tiempo_exposicion_segundos': exif.get('tiempo_exposicion_segundos'),
                'distancia_focal': exif.get('distancia_focal'),
                'modelo_camara': exif.get('modelo_camara')
            },
            'dimensiones': {
                'ancho': metadatos.get('ancho_original', 0),
                'alto': metadatos.get('alto_original', 0)
            },
            'calidad': {
                'nitidez_laplacian': calidad.get('nitidez', {}).get('laplacian', 0.0),
                'exposicion_brillo_medio': calidad.get('exposicion', {}).get('brillo_medio', 0.0),
                'ruido_snr': calidad.get('ruido', {}).get('snr', 0.0),
                'contraste_rms': calidad.get('contraste', {}).get('rms', 0.0)
            },
            'color': {
                'brillo_promedio': color.get('global', {}).get('brillo_promedio', 0.0),
                'contraste': color.get('global', {}).get('contraste', 0.0),
                'saturacion_mean': color.get('hsv', {}).get('saturation_mean', 0.0)
            },
            'saliencia': {
                'centroide_x': saliencia.get('centroide', {}).get('centroide_x_normalizado', 0.5),
                'centroide_y': saliencia.get('centroide', {}).get('centroide_y_normalizado', 0.5),
                'ratio_centro_periferia': saliencia.get('distribucion_espacial', {}).get('ratio_centro_periferia', 0.0)
            }
        }

        return resumen

In [28]:
# =============================================================================
# LECTOR DE CARACTERÍSTICAS
# =============================================================================

class LectorCaracteristicas:
    """Carga características fotográficas del Notebook 00."""

    def __init__(self, ruta_caracteristicas: Path):
        self.ruta_caracteristicas = ruta_caracteristicas
        self.ruta_json_subdir = ruta_caracteristicas / 'json'
        self.usar_subdirectorio = self.ruta_json_subdir.exists()

        if self.usar_subdirectorio:
            logger.info(f"  JSONs encontrados en subdirectorio: {self.ruta_json_subdir.relative_to(ruta_caracteristicas.parent)}")

    def obtener_ruta_json(self, foto_id: str) -> Optional[Path]:
        """Obtiene ruta al JSON de características."""
        if self.usar_subdirectorio:
            ruta_json = self.ruta_json_subdir / f"{foto_id}_caracteristicas.json"
        else:
            ruta_json = self.ruta_caracteristicas / f"{foto_id}_caracteristicas.json"

        if not ruta_json.exists():
            ruta_alternativa = self.ruta_caracteristicas / f"{foto_id}_caracteristicas.json"
            if ruta_alternativa.exists():
                return ruta_alternativa

            ruta_json_alt = self.ruta_caracteristicas / 'json' / f"{foto_id}_caracteristicas.json"
            if ruta_json_alt.exists():
                return ruta_json_alt

            return None

        return ruta_json

    def cargar(self, foto_id: str) -> Optional[Dict[str, Any]]:
        """Carga características completas de una fotografía desde JSON."""
        ruta_json = self.obtener_ruta_json(foto_id)

        if ruta_json is None:
            raise FileNotFoundError(
                f"JSON de características no encontrado para {foto_id}\n"
                f"Este es un error crítico."
            )

        try:
            with open(ruta_json, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return data

        except json.JSONDecodeError as e:
            raise ValueError(f"JSON corrupto para {foto_id}: {e}")
        except Exception as e:
            logger.error(f"Error inesperado cargando {foto_id}: {e}")
            raise

In [29]:
# =============================================================================
# LECTOR DE GROUND TRUTH
# =============================================================================

class LectorGroundTruth:
    """Carga máscaras ground truth anotadas manualmente en CVAT."""

    def __init__(self, ruta_gt: Path):
        self.ruta_gt = ruta_gt

    def obtener_ruta_npz(self, foto_id: str) -> Optional[Path]:
        """Obtiene ruta al NPZ de ground truth."""
        ruta_npz = self.ruta_gt / f"{foto_id}_gt.npz"
        return ruta_npz if ruta_npz.exists() else None

    def cargar(self, foto_id: str) -> Dict[str, Any]:
        """Carga máscara ground truth para una fotografía."""
        ruta_npz = self.obtener_ruta_npz(foto_id)

        if ruta_npz is None:
            raise FileNotFoundError(f"Ground Truth no encontrado para {foto_id}")

        try:
            data = np.load(ruta_npz)

            if 'masks' not in data:
                raise ValueError(f"NPZ de GT no contiene campo 'masks': {ruta_npz}")

            mascara = data['masks']

            if mascara.dtype == np.float32 or mascara.dtype == np.float64:
                mascara_binaria = (mascara > 0.5).astype(np.uint8)
            else:
                mascara_binaria = mascara.astype(np.uint8)

            valores_unicos = np.unique(mascara_binaria)
            if not np.all(np.isin(valores_unicos, [0, 1])):
                logger.warning(f"  GT {foto_id} tiene valores no binarios. Convirtiendo.")
                mascara_binaria = (mascara_binaria > 0).astype(np.uint8)

            area_gt = np.sum(mascara_binaria > 0)
            total_pixeles = mascara_binaria.shape[0] * mascara_binaria.shape[1]
            cobertura_pct = (area_gt / total_pixeles) * 100

            return {
                'mascara': mascara_binaria,
                'shape': mascara_binaria.shape,
                'area': int(area_gt),
                'cobertura_pct': float(cobertura_pct),
                'ruta': str(ruta_npz)
            }

        except Exception as e:
            raise RuntimeError(f"Error cargando GT para {foto_id}: {e}")

In [37]:
class EscanerConfiguraciones:
    """Escanea recursivamente toda la estructura de modelos sin límites."""

    def __init__(self, ruta_modelos: Path):
        self.ruta_modelos = ruta_modelos

    def escanear(self) -> Dict[str, List[Tuple[str, Path]]]:
        """Escanea todos los modelos y sus configuraciones recursivamente."""
        configuraciones = {}

        subdirs_modelos = [
            d for d in self.ruta_modelos.iterdir()
            if d.is_dir() and not d.name.startswith('.')
        ]

        logger.info(f"\n  Modelos encontrados: {[d.name for d in subdirs_modelos]}")

        for modelo_dir in subdirs_modelos:
            nombre_modelo = modelo_dir.name
            logger.info(f"\n  Escaneando modelo: {nombre_modelo}")

            configs_modelo = self._escanear_modelo_recursivo(modelo_dir)

            if len(configs_modelo) > 0:
                configuraciones[nombre_modelo] = configs_modelo
                logger.info(f"    ✓ Encontradas {len(configs_modelo)} configuraciones")
                for codigo, ruta in configs_modelo:
                    logger.info(f"      - {codigo}")
            else:
                logger.warning(f"    ✗ No se encontraron configuraciones válidas")

        return configuraciones

    def _escanear_modelo_recursivo(self, modelo_dir: Path) -> List[Tuple[str, Path]]:
        """Escanea COMPLETAMENTE de forma recursiva buscando configuraciones."""
        configs = []

        def buscar_recursivo(directorio: Path, partes_acumuladas: List[str]) -> None:
            """Búsqueda recursiva sin límite de profundidad."""

            # Verificar si este directorio ES una configuración válida
            tiene_mascaras = (directorio / 'mascaras').exists()
            tiene_json = (directorio / 'json').exists()
            tiene_viz = (directorio / 'visualizaciones').exists()

            # Si es configuración válida, registrarla
            if tiene_mascaras or tiene_json or tiene_viz:
                codigo = self._generar_codigo_config(
                    modelo_dir.name,
                    '_'.join(partes_acumuladas)
                )
                configs.append((codigo, directorio))
                # NO hacer return aquí - puede haber más configs dentro

            # SIEMPRE explorar subdirectorios (incluso si es config válida)
            try:
                subdirs = [
                    d for d in directorio.iterdir()
                    if d.is_dir() and not d.name.startswith('.') and d.name not in ['mascaras', 'json', 'visualizaciones']
                ]

                for subdir in subdirs:
                    nuevas_partes = partes_acumuladas + [subdir.name]
                    buscar_recursivo(subdir, nuevas_partes)
            except PermissionError:
                pass  # Ignorar directorios sin permisos

        # Iniciar búsqueda desde el directorio del modelo
        try:
            subdirs_nivel1 = [
                d for d in modelo_dir.iterdir()
                if d.is_dir() and not d.name.startswith('.')
            ]

            for subdir in subdirs_nivel1:
                buscar_recursivo(subdir, [subdir.name])
        except PermissionError:
            logger.error(f"    ✗ Sin permisos para leer: {modelo_dir}")

        return configs

    @staticmethod
    def _generar_codigo_config(nombre_modelo: str, nombre_config: str) -> str:
        """Genera código único GitHub-safe para una configuración."""
        def sanitizar(texto: str) -> str:
            texto = texto.lower()
            texto = texto.replace(' ', '_').replace('-', '_')
            texto = ''.join(c if c.isalnum() or c == '_' else '' for c in texto)
            texto = '_'.join(filter(None, texto.split('_')))
            return texto

        modelo_clean = sanitizar(nombre_modelo)
        config_clean = sanitizar(nombre_config)

        return f"{modelo_clean}_{config_clean}"

In [41]:
class IndexadorModelos:
    """Crea índice de máscaras y metadata de modelos sin cargar datos."""

    def indexar_configuracion(
        self,
        foto_id: str,
        ruta_config: Path,
        codigo_config: str
    ) -> Optional[Dict[str, Any]]:
        """Indexa una configuración de modelo para una foto."""

        # Buscar archivos que CONTENGAN el foto_id
        ruta_mascara = self._buscar_archivo_contiene(ruta_config, foto_id, ['.npz'])
        ruta_json = self._buscar_archivo_contiene(ruta_config, foto_id, ['.json'])
        ruta_viz = self._buscar_archivo_contiene(ruta_config, foto_id, ['.png', '.jpg'])

        # Si no hay ni máscara ni metadata, no indexar
        if ruta_mascara is None and ruta_json is None:
            return None

        indice = {
            'codigo_config': codigo_config,
            'ruta_mascara': str(ruta_mascara) if ruta_mascara else None,
            'ruta_metadata': str(ruta_json) if ruta_json else None,
            'ruta_visualizacion': str(ruta_viz) if ruta_viz else None
        }

        # Extraer resumen de metadata si existe
        if ruta_json:
            try:
                with open(ruta_json, 'r', encoding='utf-8') as f:
                    metadata = json.load(f)
                indice['resumen_metadata'] = self._extraer_resumen_metadata(metadata)
            except Exception as e:
                logger.debug(f"Error leyendo metadata: {e}")

        return indice

    @staticmethod
    def _buscar_archivo_contiene(
        directorio_base: Path,
        foto_id: str,
        extensiones: List[str]
    ) -> Optional[Path]:
        """
        Busca recursivamente un archivo cuyo nombre CONTIENE foto_id.

        Args:
            directorio_base: Directorio donde buscar
            foto_id: ID de la foto (ej: '_DSC0036')
            extensiones: Lista de extensiones válidas (ej: ['.npz', '.json'])

        Returns:
            Path al primer archivo encontrado o None
        """
        for archivo in directorio_base.rglob("*"):
            if archivo.is_file():
                # Verificar que el nombre CONTIENE foto_id
                if foto_id in archivo.name:
                    # Verificar extensión
                    if archivo.suffix in extensiones:
                        return archivo

        return None

    @staticmethod
    def _extraer_resumen_metadata(metadata: Dict[str, Any]) -> Dict[str, Any]:
        """Extrae resumen ligero de metadata de modelo."""
        resumen = {}

        if 'rendimiento' in metadata or 'procesamiento' in metadata:
            rendimiento = metadata.get('rendimiento', metadata.get('procesamiento', {}))
            resumen['tiempo_inferencia_ms'] = rendimiento.get('tiempo_inferencia_ms',
                                                              rendimiento.get('tiempo_total_ms', 0.0))
            resumen['gpu_pico_mb'] = rendimiento.get('gpu_pico_mb', 0.0)

        if 'metadata' in metadata:
            meta = metadata['metadata']
            resumen['modelo'] = meta.get('modelo', '')
            resumen['configuracion'] = meta.get('nombre', '')

        if 'resultados' in metadata:
            res = metadata['resultados']
            resumen['personas_detectadas'] = res.get('personas_detectadas', 0)

        return resumen

In [32]:
# =============================================================================
# INTEGRADOR DE ÍNDICES
# =============================================================================

class IntegradorIndices:
    """Integra información de todas las fuentes en índices unificados."""

    def __init__(self, config_rutas: ConfiguracionRutas):
        self.config_rutas = config_rutas

        self.lector_caracteristicas = LectorCaracteristicas(config_rutas.ruta_caracteristicas)
        self.lector_gt = LectorGroundTruth(config_rutas.ruta_gt)
        self.indexador_modelos = IndexadorModelos()

        self.escaner = EscanerConfiguraciones(config_rutas.ruta_modelos)
        self.configs_disponibles = self.escaner.escanear()

        logger.info(f"\n  RESUMEN DE CONFIGURACIONES DESCUBIERTAS:")
        total_configs = 0
        for modelo, configs in self.configs_disponibles.items():
            logger.info(f"    - {modelo}: {len(configs)} configuraciones")
            total_configs += len(configs)
        logger.info(f"  TOTAL: {total_configs} configuraciones")

    def crear_indice_fotografia(self, foto_id: str) -> IndiceFotografia:
        """Crea índice completo de una fotografía."""
        ruta_imagen = self.config_rutas.ruta_imagenes / f"{foto_id}.jpg"
        ruta_json_caract = self.lector_caracteristicas.obtener_ruta_json(foto_id)
        ruta_gt_npz = self.lector_gt.obtener_ruta_npz(foto_id)

        if ruta_json_caract is None:
            raise FileNotFoundError(f"No se encontró JSON de características para {foto_id}")

        if ruta_gt_npz is None:
            raise FileNotFoundError(f"No se encontró ground truth para {foto_id}")

        caracteristicas_completas = self.lector_caracteristicas.cargar(foto_id)
        gt_data = self.lector_gt.cargar(foto_id)

        resumen_caracteristicas = ExtractorResumenCaracteristicas.extraer_resumen(caracteristicas_completas)

        logger.info(f"    Características: OK")
        logger.info(f"    Ground Truth: OK (área={gt_data['area']}, cobertura={gt_data['cobertura_pct']:.1f}%)")

        modelos_disponibles = {}

        for nombre_modelo, configs in self.configs_disponibles.items():
            for codigo_config, ruta_config in configs:
                indice_modelo = self.indexador_modelos.indexar_configuracion(
                    foto_id, ruta_config, codigo_config
                )

                if indice_modelo is not None:
                    modelos_disponibles[codigo_config] = indice_modelo

        logger.info(f"    Modelos indexados: {len(modelos_disponibles)}")

        rutas = {
            'imagen': str(ruta_imagen),
            'caracteristicas': str(ruta_json_caract),
            'ground_truth': str(ruta_gt_npz)
        }

        ground_truth_info = {
            'shape': gt_data['shape'],
            'area': gt_data['area'],
            'cobertura_pct': gt_data['cobertura_pct']
        }

        indice = IndiceFotografia(
            foto_id=foto_id,
            rutas=rutas,
            resumen_caracteristicas=resumen_caracteristicas,
            modelos_disponibles=modelos_disponibles,
            ground_truth_info=ground_truth_info,
            timestamp_indexacion=datetime.now().isoformat()
        )

        return indice

In [33]:
# =============================================================================
# VALIDADOR DE INTEGRIDAD
# =============================================================================

class ValidadorIntegridad:
    """Valida integridad de índices creados."""

    @staticmethod
    def validar(indices: List[IndiceFotografia]) -> Dict[str, Any]:
        """Valida integridad de índices."""
        if len(indices) == 0:
            return {
                'valido': False,
                'error_critico': 'No hay índices para validar',
                'warnings': [],
                'estadisticas': {}
            }

        total_fotos = len(indices)
        fotos_con_gt = sum(1 for idx in indices if idx.ground_truth_info['area'] > 0)
        fotos_con_exif = sum(
            1 for idx in indices
            if idx.resumen_caracteristicas.get('exif', {}).get('modelo_camara') is not None
        )

        configs_todas = set()
        configs_por_modelo = {}
        configs_por_foto = []

        for indice in indices:
            configs_foto = set(indice.modelos_disponibles.keys())
            configs_todas.update(configs_foto)
            configs_por_foto.append(len(configs_foto))

            for config in configs_foto:
                modelo = config.split('_')[0]
                if modelo not in configs_por_modelo:
                    configs_por_modelo[modelo] = set()
                configs_por_modelo[modelo].add(config)

        warnings = []

        if fotos_con_gt < total_fotos:
            warnings.append(f"Solo {fotos_con_gt}/{total_fotos} fotos tienen Ground Truth")

        if len(configs_por_foto) > 0:
            config_counts = {}
            for count in configs_por_foto:
                config_counts[count] = config_counts.get(count, 0) + 1

            if len(config_counts) > 1:
                warnings.append(f"Número inconsistente de configuraciones por foto: {config_counts}")

        estadisticas = {
            'total_fotografias': total_fotos,
            'fotos_con_ground_truth': fotos_con_gt,
            'fotos_con_exif': fotos_con_exif,
            'porcentaje_con_exif': (fotos_con_exif / total_fotos * 100) if total_fotos > 0 else 0,
            'total_configuraciones_unicas': len(configs_todas),
            'configuraciones_por_modelo': {
                modelo: len(configs)
                for modelo, configs in configs_por_modelo.items()
            },
            'configuraciones_por_foto': {
                'promedio': np.mean(configs_por_foto) if len(configs_por_foto) > 0 else 0,
                'min': min(configs_por_foto) if len(configs_por_foto) > 0 else 0,
                'max': max(configs_por_foto) if len(configs_por_foto) > 0 else 0
            }
        }

        return {
            'valido': fotos_con_gt == total_fotos,
            'warnings': warnings,
            'estadisticas': estadisticas
        }

In [34]:
# =============================================================================
# PERSISTENCIA DE ÍNDICES
# =============================================================================

class PersistenciaIndices:
    """Guarda y carga sistema de índices con guardado incremental."""

    def __init__(self, ruta_analisis: Path):
        self.ruta_salida = ruta_analisis / 'fase1_integracion'
        self.ruta_salida.mkdir(exist_ok=True, parents=True)

        # Archivos
        self.ruta_indice = self.ruta_salida / 'indice_maestro.json'
        self.ruta_resumen = self.ruta_salida / 'resumen_caracteristicas.json'
        self.ruta_configs = self.ruta_salida / 'configuraciones_modelos.json'
        self.ruta_stats = self.ruta_salida / 'estadisticas_globales.json'
        self.ruta_timestamp = self.ruta_salida / 'timestamp.txt'

    def cargar_indices_existentes(self) -> Tuple[Dict, Dict]:
        """Carga índices existentes si hay."""
        indice_maestro = {}
        resumen_caract = {}

        if self.ruta_indice.exists():
            with open(self.ruta_indice, 'r', encoding='utf-8') as f:
                indice_maestro = json.load(f)

        if self.ruta_resumen.exists():
            with open(self.ruta_resumen, 'r', encoding='utf-8') as f:
                resumen_caract = json.load(f)

        return indice_maestro, resumen_caract

    def guardar_indice_fotografia(self, indice: IndiceFotografia):
        """Guarda índice de una fotografía (incremental)."""
        # Cargar existentes
        indice_maestro, resumen_caract = self.cargar_indices_existentes()

        # Actualizar con nueva foto
        indice_maestro[indice.foto_id] = indice.a_dict()
        resumen_caract[indice.foto_id] = indice.resumen_caracteristicas

        # Guardar actualizados
        with open(self.ruta_indice, 'w', encoding='utf-8') as f:
            json.dump(indice_maestro, f, indent=2, ensure_ascii=False)

        with open(self.ruta_resumen, 'w', encoding='utf-8') as f:
            json.dump(resumen_caract, f, indent=2, ensure_ascii=False)

    def guardar_configuraciones(self, configs_disponibles: Dict[str, List[Tuple[str, Path]]]):
        """Guarda configuraciones de modelos."""
        configs_serializables = {
            modelo: [(codigo, str(ruta)) for codigo, ruta in configs]
            for modelo, configs in configs_disponibles.items()
        }

        with open(self.ruta_configs, 'w', encoding='utf-8') as f:
            json.dump(configs_serializables, f, indent=2, ensure_ascii=False)

    def guardar_estadisticas(self, validacion: Dict[str, Any], total_fotos: int, total_configs: int):
        """Guarda estadísticas globales."""
        estadisticas = {
            'timestamp': datetime.now().isoformat(),
            'validacion': validacion,
            'total_fotografias': total_fotos,
            'total_configuraciones': total_configs
        }

        with open(self.ruta_stats, 'w', encoding='utf-8') as f:
            json.dump(estadisticas, f, indent=2, ensure_ascii=False)

    def guardar_timestamp(self, total_fotos: int, total_configs: int):
        """Guarda timestamp final."""
        timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        timestamp_contenido = (
            f"Fase 1 - Sistema de Índice Inteligente\n"
            f"Timestamp: {timestamp_str}\n"
            f"Fotografías: {total_fotos}\n"
            f"Configuraciones totales: {total_configs}\n"
            f"\n"
            f"FILOSOFÍA: Sistema de índice puro sin duplicación.\n"
            f"Ground truth, máscaras y características permanecen en ubicaciones originales.\n"
        )

        with open(self.ruta_timestamp, 'w', encoding='utf-8') as f:
            f.write(timestamp_contenido)

    @staticmethod
    def cargar(ruta_analisis: Path) -> Tuple[Dict, Dict, Dict]:
        """Carga sistema de índices guardado."""
        ruta_base = ruta_analisis / 'fase1_integracion'

        if not ruta_base.exists():
            raise FileNotFoundError(f"Directorio de Fase 1 no existe: {ruta_base}")

        ruta_indice = ruta_base / 'indice_maestro.json'
        if not ruta_indice.exists():
            raise FileNotFoundError(f"No se encontró índice maestro: {ruta_indice}")

        with open(ruta_indice, 'r', encoding='utf-8') as f:
            indice_maestro = json.load(f)

        ruta_resumen = ruta_base / 'resumen_caracteristicas.json'
        if not ruta_resumen.exists():
            raise FileNotFoundError(f"No se encontró resumen: {ruta_resumen}")

        with open(ruta_resumen, 'r', encoding='utf-8') as f:
            resumen_caract = json.load(f)

        ruta_stats = ruta_base / 'estadisticas_globales.json'
        if not ruta_stats.exists():
            raise FileNotFoundError(f"No se encontró estadísticas: {ruta_stats}")

        with open(ruta_stats, 'r', encoding='utf-8') as f:
            estadisticas = json.load(f)

        logger.info(f"Sistema de índices cargado desde: {ruta_base}")
        logger.info(f"  - Índice maestro: {len(indice_maestro)} fotografías")
        logger.info(f"  - Resúmenes: {len(resumen_caract)} fotografías")

        return indice_maestro, resumen_caract, estadisticas

In [35]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def ejecutar_fase1_indice_inteligente(ruta_base_tfm: str) -> Tuple[List[IndiceFotografia], Dict[str, Any]]:
    """Ejecuta la Fase 1 completa con sistema de índice inteligente."""
    logger.info("="*80)
    logger.info("FASE 1: SISTEMA DE ÍNDICE INTELIGENTE")
    logger.info("="*80)

    logger.info("\n[1/6] Descubriendo estructura del proyecto...")
    config_rutas = ConfiguracionRutas.descubrir_desde_base(ruta_base_tfm)

    config_rutas.mostrar_estructura()

    rutas_validas, errores_rutas = config_rutas.validar()
    if not rutas_validas:
        logger.error("\nErrores en la estructura de directorios:")
        for error in errores_rutas:
            logger.error(f"  - {error}")
        raise FileNotFoundError("No se pudo validar la estructura del proyecto.")

    logger.info("\n  Estructura validada correctamente")

    logger.info("\n[2/6] Escaneando fotografías en directorio...")
    imagenes = sorted(config_rutas.ruta_imagenes.glob("*.jpg"))
    fotos_ids = [img.stem for img in imagenes]

    logger.info(f"  Fotografías encontradas: {len(fotos_ids)}")

    if len(fotos_ids) == 0:
        raise FileNotFoundError(f"No se encontraron imágenes JPG en {config_rutas.ruta_imagenes}")

    if len(fotos_ids) > 0:
        logger.info(f"  Primeras fotos: {', '.join(fotos_ids[:3])}")
        if len(fotos_ids) > 3:
            logger.info(f"  ... y {len(fotos_ids) - 3} más")

    logger.info("\n[3/6] Inicializando integrador de índices...")
    integrador = IntegradorIndices(config_rutas)

    logger.info("\n[4/6] Creando índices por fotografía (GUARDADO INCREMENTAL)...")
    logger.info("-"*80)

    persistencia = PersistenciaIndices(config_rutas.ruta_analisis)

    # Guardar configuraciones una sola vez
    persistencia.guardar_configuraciones(integrador.configs_disponibles)
    logger.info(f"  Configuraciones guardadas: {persistencia.ruta_configs.name}")

    indices = []
    errores_criticos = []

    for i, foto_id in enumerate(fotos_ids, 1):
        logger.info(f"\n[{i}/{len(fotos_ids)}] Procesando: {foto_id}")

        try:
            indice = integrador.crear_indice_fotografia(foto_id)
            indices.append(indice)

            # GUARDADO INCREMENTAL
            persistencia.guardar_indice_fotografia(indice)
            logger.info(f"    ✓ Guardado incremental completado")

        except Exception as e:
            errores_criticos.append({
                'foto_id': foto_id,
                'error': str(e)
            })
            logger.error(f"  ERROR CRÍTICO: {e}")

    if len(errores_criticos) > 0:
        logger.error("\n" + "="*80)
        logger.error("ERRORES CRÍTICOS ENCONTRADOS")
        logger.error("="*80)
        for error in errores_criticos:
            logger.error(f"  - {error['foto_id']}: {error['error']}")

        raise RuntimeError(f"Se encontraron {len(errores_criticos)} errores críticos.")

    logger.info("\n[5/6] Validando integridad de índices...")
    logger.info("-"*80)

    validacion = ValidadorIntegridad.validar(indices)

    if not validacion['valido']:
        raise RuntimeError(f"Validación de integridad falló: {validacion.get('error_critico')}")

    stats = validacion['estadisticas']
    logger.info("\nEstadísticas de indexación:")
    logger.info(f"  - Total fotografías: {stats['total_fotografias']}")
    logger.info(f"  - Con Ground Truth: {stats['fotos_con_ground_truth']}")
    logger.info(f"  - Con EXIF: {stats['fotos_con_exif']} ({stats['porcentaje_con_exif']:.1f}%)")
    logger.info(f"  - Configuraciones únicas: {stats['total_configuraciones_unicas']}")
    logger.info(f"  - Configs por foto (promedio): {stats['configuraciones_por_foto']['promedio']:.1f}")

    logger.info("\nConfigs por modelo:")
    for modelo, count in stats['configuraciones_por_modelo'].items():
        logger.info(f"  - {modelo}: {count} configuraciones")

    if len(validacion['warnings']) > 0:
        logger.warning("\nAdvertencias:")
        for warning in validacion['warnings']:
            logger.warning(f"  - {warning}")

    logger.info("\n[6/6] Guardando estadísticas y timestamp final...")
    total_configs = sum(len(idx.modelos_disponibles) for idx in indices)
    persistencia.guardar_estadisticas(validacion, len(indices), total_configs)
    persistencia.guardar_timestamp(len(indices), total_configs)

    logger.info("\n" + "="*80)
    logger.info("FASE 1 COMPLETADA EXITOSAMENTE")
    logger.info("="*80)
    logger.info(f"Fotografías indexadas: {len(indices)}")
    logger.info(f"Configuraciones totales: {total_configs}")
    logger.info(f"\nÍndices guardados en: {config_rutas.ruta_analisis / 'fase1_integracion'}")
    logger.info("\nFILOSOFÍA: Sistema de índice puro sin duplicación.")
    logger.info("Ground truth, máscaras y características permanecen en ubicaciones originales.")

    return indices, validacion

In [42]:
# =============================================================================
# PUNTO DE ENTRADA
# =============================================================================

if __name__ == "__main__":

    RUTA_BASE_TFM = "/content/drive/MyDrive/TFM"

    try:
        indices_fotos, validacion = ejecutar_fase1_indice_inteligente(RUTA_BASE_TFM)

        print("\n" + "="*80)
        print("RESUMEN DE INDEXACIÓN")
        print("="*80)
        print(f"Fotografías indexadas: {len(indices_fotos)}")
        print(f"Validación: {'EXITOSA' if validacion['valido'] else 'FALLIDA'}")

        if len(indices_fotos) > 0:
            print(f"\nEjemplo de índice (primera foto):")
            ejemplo = indices_fotos[0]
            print(f"  - ID: {ejemplo.foto_id}")
            print(f"  - GT disponible: {ejemplo.ground_truth_info['area'] > 0}")
            print(f"  - Dimensiones: {ejemplo.resumen_caracteristicas['dimensiones']}")
            print(f"  - ISO: {ejemplo.resumen_caracteristicas['exif']['iso']}")
            print(f"  - Modelos indexados: {len(ejemplo.modelos_disponibles)}")

            modelos_unicos = list(set(k.split('_')[0] for k in ejemplo.modelos_disponibles.keys()))
            print(f"  - Modelos: {modelos_unicos}")
    except Exception as e:
      print(f"\nERROR DURANTE EJECUCIÓN: {e}")
      import traceback
      traceback.print_exc()
      raise

[2025-11-15 18:05:03] INFO - ================================================================================
[2025-11-15 18:05:03] INFO - FASE 1: SISTEMA DE ÍNDICE INTELIGENTE
[2025-11-15 18:05:03] INFO - ================================================================================
[2025-11-15 18:05:03] INFO - 
[1/6] Descubriendo estructura del proyecto...
[2025-11-15 18:05:03] INFO - Escaneando estructura en: /content/drive/MyDrive/TFM
[2025-11-15 18:05:03] INFO - 
Estructura del proyecto:
[2025-11-15 18:05:03] INFO -   Base:             /content/drive/MyDrive/TFM
[2025-11-15 18:05:03] INFO -   Imágenes:         0_Imagenes
[2025-11-15 18:05:03] INFO -   Características:  1_Caracteristicas
[2025-11-15 18:05:03] INFO -   Ground Truth:     0_Imagenes_CVAT/ground_truth_masks
[2025-11-15 18:05:03] INFO -   Modelos:          2_Modelos
[2025-11-15 18:05:03] INFO -   Análisis:         3_Analisis
[2025-11-15 18:05:03] INFO - 
  Estructura validada correctamente
[2025-11-15 18:05:03] INFO -


RESUMEN DE INDEXACIÓN
Fotografías indexadas: 20
Validación: EXITOSA

Ejemplo de índice (primera foto):
  - ID: _DSC0023
  - GT disponible: True
  - Dimensiones: {'ancho': 4016, 'alto': 6016}
  - ISO: 200
  - Modelos indexados: 127
  - Modelos: ['mask2former', 'sam2', 'yolov8', 'oneformer', 'bodypix']
